# PyHEP Numba tutorial on February 3, 2021

## Simple stuff

Even if you've used Numba before, let's start with the basics.

In fact, let's start with NumPy itself.

### NumPy

In [1]:
import numpy as np

NumPy accelerates Python code by replacing loops in Python's virtual machine (with type-checks at runtime) with precompiled loops that transform arrays into arrays.

In [2]:
data = np.arange(1000000)
data

array([     0,      1,      2, ..., 999997, 999998, 999999])

In [3]:
%%timeit

output = np.empty(len(data))

for i, x in enumerate(data):
    output[i] = x**2

346 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit

output = data**2

695 µs ± 8.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


But if you have to compute a complex formula, NumPy would have to _make an array for each intermediate step_.

(There are tricks for circumventing this, but we won't get into that.)

In [5]:
energy = np.random.normal(100, 10, 1000000)
px = np.random.normal(0, 10, 1000000)
py = np.random.normal(0, 10, 1000000)
pz = np.random.normal(0, 10, 1000000)

In [6]:
%%timeit

mass = np.sqrt(energy**2 - px**2 - py**2 - pz**2)

6.56 ms ± 498 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The above is equivalent to

In [7]:
%%timeit

tmp1 = energy**2
tmp2 = px**2
tmp3 = py**2
tmp4 = pz**2
tmp5 = tmp1 - tmp2
tmp6 = tmp5 - tmp3
tmp7 = tmp6 - tmp4
mass = np.sqrt(tmp7)

8.02 ms ± 403 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Numba

(I always mistype "numba" as "numpy"...)

In [8]:
import numba as nb

Numba lets us compile a function to compute a whole formula in one step.

```python
@nb.jit
```

means "JIT-compile" and

```python
@nb.njit
```

means "really JIT-compile" because the original has a fallback mode that's getting deprecated. If we're using Numba at all, we don't want it to fall back to ordinary Python.

In [9]:
@nb.njit
def compute_mass(energy, px, py, pz):
    mass = np.empty(len(energy))
    for i in range(len(energy)):
        mass[i] = np.sqrt(energy[i]**2 - px[i]**2 - py[i]**2 - pz[i]**2)
    return mass

The `compute_mass` function is now "ready to be compiled." It will be compiled when we give it arguments, so that it can propagate types.

In [10]:
compute_mass(energy, px, py, pz)

array([102.52544721,  99.6333991 , 109.71246135, ...,  86.76466324,
        96.45253913,  99.89977107])

In [11]:
%%timeit

mass = compute_mass(energy, px, py, pz)

2.7 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


_(Note to self: show `fastmath` and `parallel`.)_

## Numba performance mistakes

What's wrong with the following?

In [12]:
@nb.njit
def compute_mass_i(energy_i, px_i, py_i, pz_i):
    return np.sqrt(energy_i**2 - px_i**2 - py_i**2 - pz_i**2)

compute_mass_i(energy[0], px[0], py[0], pz[0])

102.52544721062226

In [13]:
%%timeit

mass = np.empty(len(energy))
for i in range(len(energy)):
    mass[i] = compute_mass_i(energy[i], px[i], py[i], pz[i])

789 ms ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


What do you think about this one?

In [14]:
@nb.njit
def compute_mass_arrays(energy, px, py, pz):
    return np.sqrt(energy**2 - px**2 - py**2 - pz**2)

compute_mass_arrays(energy, px, py, pz)

array([102.52544721,  99.6333991 , 109.71246135, ...,  86.76466324,
        96.45253913,  99.89977107])

In [15]:
%%timeit

mass = compute_mass_arrays(energy, px, py, pz)

2.35 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


_(Note to self: show `nb.vectorize`.)_

## Types matter!

### Dynamically typed programs that are not statically typed

Much of Python's flexibility comes from the fact that it does not need to know the types of all variables before it starts to run.

That dynamism makes it easier to express complex logic (what I call "bookkeeping"), but it is a hurdle for speed. Dynamic typing was the first thing to go in Didier Verna's _How to make LISP go faster than C_.

In [16]:
def perfectly_valid_script(data):
    output = np.empty(len(data))
    for i, group in enumerate(data):
        minimum = "nothing yet"
        for x in group:
            if minimum == "nothing yet":
                minimum = x
            elif x < minimum:
                minimum = x
        output[i] = minimum
    return output

In [17]:
data = np.random.normal(2, 1, (100000, 3))
data

array([[3.15127999, 0.39407033, 2.40792158],
       [2.4719327 , 1.951657  , 1.19690607],
       [1.50390663, 2.6622091 , 1.33609662],
       ...,
       [1.64232371, 1.97757591, 3.66571519],
       [0.45069564, 1.97901781, 0.73951216],
       [4.11026688, 2.44254492, 3.55893654]])

In [18]:
perfectly_valid_script(data)

array([0.39407033, 1.19690607, 1.33609662, ..., 1.64232371, 0.45069564,
       2.44254492])

In [19]:
invalid_for_numba = nb.njit(perfectly_valid_script)

In [20]:
invalid_for_numba(data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function lt>) found for signature:
 
 >>> lt(float64, Literal[str](nothing yet))
 
There are 22 candidate implementations:
[1m  - Of which 20 did not match due to:
  Overload of function 'lt': File: <numerous>: Line N/A.
    With argument(s): '(float64, unicode_type)':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Operator Overload in function 'lt': File: unknown: Line unknown.
    With argument(s): '(float64, unicode_type)':[0m
[1m   No match for registered cases:
    * (bool, bool) -> bool
    * (int8, int8) -> bool
    * (int16, int16) -> bool
    * (int32, int32) -> bool
    * (int64, int64) -> bool
    * (uint8, uint8) -> bool
    * (uint16, uint16) -> bool
    * (uint32, uint32) -> bool
    * (uint64, uint64) -> bool
    * (float32, float32) -> bool
    * (float64, float64) -> bool[0m
[0m
[0m[1mDuring: typing of intrinsic-ca

How can we fix it up?

### What is "type unification?"

Consider the following:

In [21]:
def another_valid_script(data):
    output = np.empty(len(data))
    for i, group in enumerate(data):
        total = np.sum(group)
        if total < 0:
            return "we don't like negative sums"
        else:
            output[i] = total
    return output

In [22]:
another_valid_script(data[:10])

array([5.95327189, 5.62049577, 5.50221235, 5.64634391, 4.34770178,
       6.03129462, 3.79146602, 6.91879592, 9.54459418, 4.55619239])

In [23]:
invalid_for_numba = nb.njit(another_valid_script)

In [24]:
invalid_for_numba(data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mCan't unify return type from the following types: Literal[str](we don't like negative sums), array(float64, 1d, C)
[1mReturn of: IR name '$54return_value.2', type 'Literal[str](we don't like negative sums)', location: [1m
File "<ipython-input-21-dade312fbc5a>", line 6:[0m
[1mdef another_valid_script(data):
    <source elided>
        if total < 0:
[1m            return "we don't like negative sums"
[0m            [1m^[0m[0m[0m
[1mReturn of: IR name '$68return_value.1', type 'array(float64, 1d, C)', location: [1m
File "<ipython-input-21-dade312fbc5a>", line 9:[0m
[1mdef another_valid_script(data):
    <source elided>
            output[i] = total
[1m    return output
[0m    [1m^[0m[0m[0m[0m

Does it matter whether we limit `data` to the first 10 elements?

How can we fix it up?

### Avoid lists and dicts

Although Numba developers are doing a lot of work on supporting Python `list` and `dict` (of identically typed contents), I find it to be too easy to run into unsupported cases. The main problem is that their contents _must_ be typed, but the Python language didn't have ways to express types.

(Yes, Python has type annotations now, but they're not fully integrated into Numba yet.)

Let's start with something that works and make small changes to it.

In [48]:
@nb.njit
def output_a_list(data):
    output = []
    for group in data:
        total = 0.0
        for x in group:
            total += x
        output.append(total)
    return output

In [51]:
data = np.random.normal(2, 1, (10, 3))
data

array([[ 0.95598489,  3.31140722,  1.79358308],
       [ 2.27913826,  0.81978631,  2.79319435],
       [ 1.92183806,  3.8611939 ,  2.04272215],
       [ 0.62488518,  3.48649249,  1.44912967],
       [ 2.12085548,  3.66397727, -0.0990048 ],
       [ 4.02775381,  1.55623024,  2.93972499],
       [ 1.65313006,  1.43999003,  1.94078311],
       [ 3.482674  , -0.1964865 ,  2.04161286],
       [ 2.010127  ,  1.10942775,  2.89436227],
       [ 1.46269584,  3.7490703 ,  3.04671471]])

In [52]:
output_a_list(data)

[6.060975199235453,
 5.892118921096655,
 7.825754101258172,
 5.560507332144437,
 5.685827949831095,
 8.52370904297949,
 5.033903196524288,
 5.327800353699507,
 6.013917015164047,
 8.258480847563252]

### Closures versus arguments

In Python, you can create functions at runtime, and these functions can reference data defined outside of the function.

In [28]:
accumulate = nb.typed.List([])

def yet_another_valid_script(data):
    for group in data:
        total = 0.0
        for x in group:
            total += x
        accumulate.append(total)

In [29]:
accumulate

ListType[Undefined]([])

In [30]:
yet_another_valid_script(data)

In [31]:
accumulate

ListType[float64]([5.5682422577706365, 4.667006503512932, 6.000827124822459, 7.662138716429409, 2.9710870225622283, 7.996927518540405, 5.454419989342256, 5.179224328118463, 6.1410842988971, 5.316298015049456])

In [32]:
try_it_in_numba = nb.njit(yet_another_valid_script)

In [33]:
try_it_in_numba(data)

TypingError: Failed in nopython mode pipeline (step: ensure IR is legal prior to lowering)
[1mThe use of a ListType[float64] type, assigned to variable 'accumulate' in globals, is not supported as globals are considered compile-time constants and there is no known way to compile a ListType[float64] type as a constant.
[1m
File "<ipython-input-28-725ca9e5f929>", line 8:[0m
[1mdef yet_another_valid_script(data):
    <source elided>
            total += x
[1m        accumulate.append(total)
[0m        [1m^[0m[0m
[0m

In [ ]:
accumulate

### Functions calling functions

The `@nb.njit` decorator brackets a region of code to make fast. But that doesn't mean you need to write monolithic functions; JIT'ed functions can be rapidly executed within other JIT'ed functions.

In [70]:
@nb.njit
def find_minimum(group):
    out = None
    for x in group:
        if out is None:
            out = x
        elif x < out:
            out = x
    return out

In [71]:
@nb.njit
def run_over_all(data):
    out = np.empty(len(data))
    for i, group in enumerate(data):
        out[i] = find_minimum(group)
    return out

In [72]:
data = np.random.normal(2, 1, (1000000, 3))
data

array([[1.36017057, 1.30274139, 1.27922324],
       [1.43282889, 0.96010922, 2.39008941],
       [3.11312149, 3.75215365, 2.04939373],
       ...,
       [0.89504617, 1.55006642, 2.70655948],
       [2.3303123 , 1.21097414, 1.78995369],
       [1.68926693, 2.13946256, 0.29343878]])

In [73]:
run_over_all(data)

array([1.27922324, 0.96010922, 2.04939373, ..., 0.89504617, 1.21097414,
       0.29343878])

In [74]:
%%timeit

run_over_all(data)

13.5 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Functional functions

A JIT'ed function can also be _passed into_ a JIT'ed function as an argument. Same constraints apply.

In [75]:
@nb.njit
def run_over_all(do_what, data):
    out = np.empty(len(data))
    for i, group in enumerate(data):
        out[i] = do_what(group)
    return out

In [76]:
run_over_all(find_minimum, data)

array([1.27922324, 0.96010922, 2.04939373, ..., 0.89504617, 1.21097414,
       0.29343878])

In [77]:
%%timeit

run_over_all(find_minimum, data)

13.7 ms ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


What about functions _returning_ functions?

In [80]:
@nb.njit
def function_returning_a_function():
    def what_the_heck(x):
        return x + 1
    return what_the_heck

In [82]:
f = function_returning_a_function()
f

CPUDispatcher(<function g.<locals>.f at 0x7f5dd97760d0>)

In [83]:
f(10)

11

But the reason you might want to do this—to make a closure—doesn't work, so I consider this as a curiosity.

In [84]:
@nb.njit
def function_returning_a_function(y):
    def what_the_heck(x):
        return x + y
    return what_the_heck

In [87]:
function_returning_a_function(1)

TypingError: Failed in nopython mode pipeline (step: convert make_function into JIT functions)
[1mCannot capture the non-constant value associated with variable 'y' in a function that will escape.
[1m
File "<ipython-input-84-d716b8b1def2>", line 3:[0m
[1mdef function_returning_a_function(y):
[1m    def what_the_heck(x):
[0m    [1m^[0m[0m
[0m

### What about classes?

Numba has a whole system for annotating classes to be used in JIT'ed functions: `@nb.jitclass` ([see documentation](https://numba.pydata.org/numba-doc/latest/user/jitclass.html)).

However, I rarely use it—you find yourself having to annotate more and more until it doesn't look like Python. The value of Numba is that you can develop quickly without leaving the Python environment. When you're doing something so complex as to need classes, you might want to write a C++ program bound to Python with [pybind11](https://pybind11.readthedocs.io/) or ROOT.

But here's a simple one:

In [174]:
@nb.experimental.jitclass([
    ("E", nb.float64),
    ("px", nb.float64),
    ("py", nb.float64),
    ("pz", nb.float64),
])
class Lorentz:
    def __init__(self, E, px, py, pz):
        self.E = E
        self.px = px
        self.py = py
        self.pz = pz

    @property
    def mass(self):
        return np.sqrt(self.E**2 - self.px**2 - self.py**2 - self.pz**2)

@nb.njit
def add_Lorentz(one, two):
    return Lorentz(one.E + two.E, one.px + two.px, one.py + two.py, one.pz + two.pz)

In [179]:
objects = nb.typed.List([Lorentz(E, px, py, pz) for E, px, py, pz in zip(
    np.random.normal(100, 10, 10000),
    np.random.normal(0, 10, 10000),
    np.random.normal(0, 10, 10000),
    np.random.normal(0, 10, 10000),
)])

In [180]:
@nb.njit
def calculate_masses(one, two):
    out = np.empty(len(one))
    for i in range(len(one)):
        out[i] = add_Lorentz(one[i], two[i]).mass
    return out

In [182]:
calculate_masses(objects[:5000], objects[5000:])

array([187.05676346, 175.6921173 , 197.39367367, ..., 188.05724023,
       177.12639467, 176.38163678])

It works, but small deviations from this example hit pickling errors (because Numba passes class objects into low-level ↔ Python conversions by pickling them) and other errors deep in the Numba internals.

Iteration over objects easier (and faster) with Awkward Array (see below), which will have Lorentz vector objects defined by the [Vector](https://github.com/scikit-hep/vector) library (in development).

## Specializing functions

The biggest strength of JIT-compilers is that you can generate the code to compile after you know a lot about the problem.

In C++, this is the realm of templates and compile-time metaprogramming. In Numba, it can be an f-string:

In [187]:
args = [
    np.random.normal(0, 1, 1000000),
    np.random.normal(0, 1, 1000000),
    np.random.normal(0, 1, 1000000),
    np.random.normal(0, 1, 1000000),
    np.random.normal(0, 1, 1000000),
]

code = f"""
@nb.vectorize
def sum_in_quadrature({", ".join("p%d" % i for i in range(len(args)))}):
    # compiled for exactly {len(args)} arguments
    return np.sqrt({" + ".join("p%d**2" % i for i in range(len(args)))})
"""

print(code)


@nb.vectorize
def sum_in_quadrature(p0, p1, p2, p3, p4):
    # compiled for exactly 5 arguments
    return np.sqrt(p0**2 + p1**2 + p2**2 + p3**2 + p4**2)



In [188]:
exec(code)

In [189]:
sum_in_quadrature(*args)

array([2.8403486 , 2.27741088, 1.69199654, ..., 1.46661263, 1.5076158 ,
       2.41836881])

The point is that you have the full power of Python at your disposal to examine your data and generate code before compiling it, and you don't have to hack together scripts to make source code files before you can use them.

Numba has a `@nb.generated_jit` decorator for specializing a function for specific types.

In [194]:
@nb.generated_jit
def add_unmasked(data, mask):
    # At this level, data and mask are Numba *types* in plain Python.
    if isinstance(mask, nb.types.NoneType):
        def implementation(data, mask):
            # At this level, data and mask are *values* in a JIT'ed context.
            return np.sum(data)
        return implementation
    else:
        def implementation(data, mask):
            # The names "data" and "mask" have to match the outer function.
            result = 0
            for d, m in zip(data, mask):
                if not m:
                    result += d
            return result
        return implementation

In [208]:
data = np.random.normal(0, 1, 1000000)
mask = np.random.randint(0, 2, 1000000).view(np.bool_)
data, mask

(array([ 1.04332867,  0.24926463, -0.19549085, ..., -3.92769812,
        -1.55297017, -1.2654532 ]),
 array([False, False, False, ..., False, False, False]))

In [209]:
add_unmasked(data, mask)

321.6181910361945

In [210]:
add_unmasked(data, None)

-231.6161556741977

## Debugging Numba JIT'ed functions

You might argue that my examples are too simple, but that's the best way to use Numba: by merging complex logic into a procedure bit by bit. You only benefit from the "JIT" aspect if you're building it up interactively—in a Python terminal, iPython, or a Jupyter notebook.

**If your workflow is "edit script, save, run from the beginning, edit script again," then you might as well be using a non-JIT compiler, like C++.**

You can figure out how to write a function in one environment and then copy it into the script you'll be submitting to the GRID, you know.

**Recommendations:**

   * If you're still figuring out the logic of what you want to compute, write pure Python functions.
   * If you have an algorithm and want to make it fast, enclose it in a function or a few functions and put `@nb.njit` at the tops of just those functions.
   * If that doesn't work, break it down into small pieces and try to `@nb.njit` each one individually.
   * There's [documentation on using Numba in gdb](https://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#debugging-jit-compiled-code-with-gdb), but note that `print` works:

In [211]:
@nb.njit
def do_stuff(data):
    for group in data:
        print(group)

In [213]:
data = np.random.normal(0, 1, (1000000, 3))
data

array([[ 0.86424368, -1.35816304, -0.51777131],
       [-1.51917545,  0.08586805,  0.94563962],
       [-0.35078942,  1.62167214, -0.93713669],
       ...,
       [-1.65014699, -1.04484009,  0.95014528],
       [ 0.28536464,  1.41487192, -0.14499178],
       [ 0.75161761,  1.04365722,  0.17627768]])

In [214]:
do_stuff(data[:10])    # not too many!

[ 0.86424368 -1.35816304 -0.51777131]
[-1.51917545  0.08586805  0.94563962]
[-0.35078942  1.62167214 -0.93713669]
[-1.2111678   0.65547725  0.57855377]
[ 0.96334237 -1.34371725  0.10681792]
[-0.3859955   0.14630824 -0.30624752]
[0.81276506 1.30407401 2.56914148]
[ 0.66880957 -1.21955263 -0.84532021]
[-0.15594011 -0.45887374  0.90774915]
[ 2.75730885 -0.2843357   0.21809643]


Personally, I wouldn't bother with gdb. I'd just use `print`.

## Awkward Arrays in Numba

